# Homework №3

### <center>Student: *Chernyshev Daniil*</center>

# Description

1. Those how are making projects on artilces: half of you need to choose co-authorship graph, others - citation (but consider citation graph as undirected for simplicity)
2. Use weighted graph so that it would give you ability to calculate embeddings in more appropriate way

### Guidelines:

1. Initiallize your classification set as follows:
    * Determine training and testing intervals on your time domain (for example, take a period $2000$-$2014$ as training period and $2015$-$2018$ as testing period)
    * Pick pairs of nodes that **have appeared during training interval** but **had no links** during it
    * These pairs form **positive** or **negative** examples depending on whether they have formed coauthorships **during the testing interval**
    * You have arrived to binary classification problem.
2. Construct feature space:
    * Use at least 2 features based on neighborhood 
    * Use at least 2 fetures based on shortest path
    * Use embedding representation of nodes' pairs (for example, node2vec)
    * Use idea of time series features (with time lag)
    * Use idea of change-point detection
3. Choose at least $3$ classification algorithms and compare them in terms of Accuracy, Precision, Recall, F-Score (for positive class) and Mean Squared Error. Use k-fold cross-validation and average your results

In [3]:
%config Completer.use_jedi = False

In [4]:
from download import *

In [9]:
import networkx as nx
import gc
import csv


In [6]:
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
dim=32

# Build Train (check classification first!)

First of all we need to construct the training dataset. The features are folowing:
* Jaccard coefficient
* Resource allocation index
* Jaccard coefficient lag
* Resoure allocation index lag
* Distance
* Inverse distance - this defines the likelyhood of link
* Density - describes changes in graph structure
* Node2vec Hadamard embedding (edge embedding in node2vec)

Node2vec is slow algorithm, so to deal with its performance we should somehow shrink our graph for current period. It can be done by leaving only connected components that contain at least one node for which target is defined (which means node is present in future period). Since we have not much data on articles it is expected to have low positive class rate.

In [1]:
def buildfet(P, C, F, dim=16):
    fil=C.nodes & F.nodes
    lfil=C.nodes & P.nodes
    
    cc=nx.connected_component_subgraphs(C)
    Gfil=nx.Graph()
    for i in cc:
        if fil & i.nodes:
            Gfil=nx.compose(Gfil, i)
    
    ne=list(nx.non_edges(Gfil))
    
    #filter
    nef=[i for i in ne if i[0] in fil and i[1] in fil]
    lnef=[i for i in nef if i[0] in lfil and i[1] in lfil]
    print(len(nef))
    jc=nx.jaccard_coefficient(Gfil, nef)
    ra=nx.resource_allocation_index(Gfil, nef)
    
    #lag
    ljc=nx.jaccard_coefficient(P, lnef)
    lra=nx.resource_allocation_index(P, lnef)
    
    den=nx.density(Gfil)
    
    node2vec = Node2Vec(Gfil, dimensions=dim, walk_length=30, num_walks=200, workers=1)
    model = node2vec.fit(window=10, min_count=1, batch_words=4, workers=8)
    edges_embs = HadamardEmbedder(keyed_vectors=model.wv)
    kv=edges_embs.as_keyed_vectors()
    data=[]
    lj=next(ljc)
    lr=next(lra)
    for j,r,n in zip(jc, ra, nef):
        a=n[0]
        b=n[1]
        try:
            d=nx.astar_path_length(Gfil, a, b)
            ad=1/d
        except:
            d=999999999
            ad=0
        if(lj!=None and a==lj[0] and b==lj[1]):
            pj=lj
            try:
                lj=next(ljc)
            except:
                lj=None
        else:
            pj=(None,None,None)
        if(lr!=None and a==lr[0] and b==lr[1]):
            pr=lr
            try:
                lr=next(lra)
            except:
                lr=None
        else:
            pr=(None,None,None)
        target=b in F[a]
        data.append((target, a, b, j[2],r[2], pj[2], pr[2], d, ad, den) + tuple(edges_embs[(str(a), str(b))]))
    return data

In [19]:
periods=[(2010, 2011), (2013, 2013), (2014,2014), (2015, 2015), (2016, 2016)]

In [20]:
res=[]
for i in range(1,len(periods)-1):
    P=nx.Graph()
    C=nx.Graph()
    F=nx.Graph()
    P.add_weighted_edges_from(getAutGraph(*periods[i-1]))
    C.add_weighted_edges_from(getAutGraph(*periods[i]))
    F.add_weighted_edges_from(getAutGraph(*periods[i+1]))
    rda=buildfet(P,C,F)
    print(len(rda))
    res+=rda
    gc.collect()

Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████████| 200/200 [04:10<00:00,  1.23s/it]
Generating edge features: 100%|██████████████████████████████████████████| 733866/733866.0 [00:03<00:00, 221633.18it/s]


11454


Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████████| 200/200 [02:28<00:00,  1.36it/s]
Generating edge features: 100%|██████████████████████████████████████████| 211575/211575.0 [00:00<00:00, 215918.59it/s]


14051


Generating walks (CPU: 1): 100%|█████████████████████████████████████████████████████| 200/200 [03:47<00:00,  1.16s/it]
Generating edge features: 100%|██████████████████████████████████████████| 752151/752151.0 [00:03<00:00, 212881.92it/s]


30058


### This gives us training period from 2013 to 2015 (2016 is considered as future)

In [38]:

with open('./tdata1.csv', 'w', encoding="utf8") as csvoutput:
    writer = csv.writer(csvoutput, lineterminator='\n')
    writer.writerows(res)

# Build Test

We build for 2017

In [ ]:
tper=[('2016', '2016'),('2017','2017'),('2018','2019')]
test=[]
for i in range(1,len(tper)-1):
    P=nx.Graph()
    C=nx.Graph()
    F=nx.Graph()
    P.add_weighted_edges_from(getAutGraph(*tper[i-1]))
    C.add_weighted_edges_from(getAutGraph(*tper[i]))
    F.add_weighted_edges_from(getAutGraph(*tper[i+1]))
    rda=buildfet(P,C,F)
    print(len(rda))
    test+=rda
    gc.collect()

In [ ]:
with open('./test22.csv', 'w', encoding="utf8") as csvoutput:
    writer = csv.writer(csvoutput, lineterminator='\n')
    writer.writerows(test)

# Classification

First, lets load our precomputed datasets

In [10]:
with open('./tdata1.csv','r', encoding="utf8") as csvinput:
        reader = csv.reader(csvinput)
        data = list(reader)

In [11]:
import pandas as pd

In [12]:
cols=['Target','A', 'B', 'Jaccard', 'Resource_allocation', 'Jaccard_lag1', 'Resource_alloc_lag1', 'Distance', 'AntiDistance','Density'] + ['hadamard_'+str(i) for i in range(16)]


In [13]:
df=pd.DataFrame(data, columns=cols).convert_objects(convert_numeric=True).fillna(0).replace('False', False).replace('True', True)

C:\Users\Orbis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [14]:
df.head()

,Target,A,B,Jaccard,Resource_allocation,Jaccard_lag1,Resource_alloc_lag1,Distance,AntiDistance,Density,...,hadamard_6,hadamard_7,hadamard_8,hadamard_9,hadamard_10,hadamard_11,hadamard_12,hadamard_13,hadamard_14,hadamard_15
0,False,53879453700,7004454931,0.00000,0.00000,0.0,0.0,999999999,0.000000,0.025373,...,3.794715,7.450766,0.313761,-5.207485,8.924924,3.738052,4.559178,1.548265,-7.566571,-7.101907
1,False,53879453700,57204744255,0.00000,0.00000,0.0,0.0,3,0.333333,0.025373,...,-4.095002,-11.695877,0.156034,7.434962,-0.715112,0.135229,40.969475,0.575845,4.850762,1.741908
2,False,53879453700,7202300001,0.00813,0.00641,0.0,0.0,2,0.500000,0.025373,...,-2.293894,-2.871058,0.276012,1.963976,7.659011,1.409847,17.247793,0.089201,8.318015,4.951419
3,False,53879453700,7401738392,0.00000,0.00000,0.0,0.0,999999999,0.000000,0.025373,...,-0.106118,26.749517,0.449816,-3.809829,13.804201,3.075871,-18.389505,-0.684995,-17.652903,-3.019060
4,False,53879453700,7004262579,0.00000,0.00000,0.0,0.0,999999999,0.000000,0.025373,...,-4.454909,27.315493,0.790719,-8.232331,2.634415,4.641665,31.488245,-0.546559,0.624052,9.855749


In [16]:
with open('./test1.csv','r', encoding="utf8") as csvinput:
        reader = csv.reader(csvinput)
        test = list(reader)

In [17]:
te=pd.DataFrame(test, columns=cols).convert_objects(convert_numeric=True).fillna(0).replace('False', False).replace('True', True)

C:\Users\Orbis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


## Random forest

In [18]:
from sklearn.ensemble import RandomForestClassifier as rf

In [19]:
cl=rf(n_estimators=100)

In [20]:
cl.fit(df[cols[3:]], df['Target'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
cl.feature_importances_

array([0.0138881 , 0.01120263, 0.        , 0.        , 0.00269907,
       0.00319414, 0.01214652, 0.08655407, 0.06740997, 0.05960492,
       0.05017548, 0.06017548, 0.0550288 , 0.07073436, 0.06109231,
       0.05649842, 0.06525971, 0.05445577, 0.05194015, 0.05613828,
       0.04165168, 0.05394132, 0.06620882])

In [22]:
rfr=cl.predict(te[cols[3:]])

## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier as kn

In [24]:
kcl=kn(n_neighbors=15)

In [25]:
kcl.fit(df[cols[3:]], df['Target'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=15, p=2,
           weights='uniform')

In [26]:
knnr=kcl.predict(te[cols[3:]])

## SVM

In [27]:
from sklearn.svm import SVC as svm

In [28]:
scl=svm()

In [29]:
scl.fit(df[cols[3:]], df['Target'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
svmr=scl.predict(te[cols[3:]])

## Compare

In [56]:
from sklearn.metrics import precision_score as prs
from sklearn.metrics import f1_score as f1s
from sklearn.metrics import recall_score as rs
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import confusion_matrix
import numpy as np

In [52]:
tocal=[rfr,knnr,svmr]
calname=['Random Forest', 'KNN', 'SVM']

In [59]:
calculated=[]
for i in tocal:
    calculated.append((prs(te['Target'],i),rs(te['Target'],i),f1s(te['Target'],i),acs(te['Target'],i), mse(te['Target'].astype(np.float32),i.astype(np.float32))))

C:\Users\Orbis\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Orbis\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [60]:
cmp=pd.DataFrame(calculated, columns=['Precision', 'Recall', 'F1_Score', 'Accuracy', 'MSE'],index=calname)

In [61]:
cmp

,Precision,Recall,F1_Score,Accuracy,MSE
Random Forest,0.0,0.0,0.0,0.993204,0.006796
KNN,0.0,0.0,0.0,0.993204,0.006796
SVM,0.0,0.0,0.0,0.993204,0.006796


# Something is strange. Lets take a look at positive class rates

## In test

In [41]:
len(te[te['Target']])/len(te)

0.006795922446532081

Very low...

## In train

In [63]:
len(df[df['Target']])/len(df)

0.00475136331731548

Even lower!

## And predictions are...

In [74]:
CM = confusion_matrix(te['Target'], np.mean([rfr,knnr,svmr], axis=0))

TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]

In [75]:
print(CM)

[[14907     0]
 [  102     0]]


So basicaly, all algorithms are predicting only 0, which is right since the probability of 1 in training dataset is less than 1%. This means that having link is an anomalous event. Unfortunately, this gives us no posibillity to compare the actual accuracy of this classification problem. 

If we had more data, the graph would be more connected and the author cooperation would be more likely event. However, generating training dataset would be computationally expensive and would take weeks to build it (5k node coauthorship graph already results in about 0.5mil unlinked node pairs which requires about 6 hours to process). But maybe this is just python performance issue and using efficient languages like c++ would solve the challenge.